<a href="https://colab.research.google.com/github/dp-08/Gen-AI/blob/main/Image_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q diffusers transformers accelerate safetensors torch --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 111.5 MB/s eta 0:00:00


In [2]:
pip install ftfy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.7 MB/s eta 0:00:00


In [3]:
import torch
from diffusers import PixArtAlphaPipeline

/usr/local/lib/python3.12/dist-packages/torch_xla/experimental/gru.py:113: SyntaxWarning: invalid escape sequence '\_'
  * **h_n**: tensor of shape :math:`(D * \text{num\_layers}, H_{out})` or


In [4]:
model_id = "PixArt-alpha/PixArt-XL-2-512x512"
dtype = torch.float16 if torch.cuda.is_available() else torch.float32

pipe = PixArtAlphaPipeline.from_pretrained(
    model_id,
    torch_dtype=dtype,
) # Fix the missing closing parenthesis here

device = "cuda" if torch.cuda.is_available() else "cpu"

pipe = pipe.to(device) # Assign the result back to 'pipe'

#Optional: enable faster attention if available
if device == "cuda":
    pipe.enable_model_cpu_offload() # works well on limited VRAM
    pipe.enable_vae_slicing()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/452 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

text_encoder/model-00001-of-00004.safete(…):   0%|          | 0.00/4.99G [00:00<?, ?B/s]

text_encoder/model-00002-of-00004.safete(…):   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

transformer/diffusion_pytorch_model.safe(…):   0%|          | 0.00/2.45G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Some weights of the model checkpoint at /root/.cache/huggingface/hub/models--PixArt-alpha--PixArt-XL-2-512x512/snapshots/50f702106901db6d0f8b67eb88e814c56ded2692/transformer were not used when initializing PixArtTransformer2DModel: 
 ['caption_projection.y_embedding']
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
prompt = ("A beautiful, happy, fluffy golden retriever puppy. The puppy has a playful expression, shining deep brown eyes, and is sitting in a sunny garden filled with green grass and wildflowers. The photo is a close-up, high-quality, professional studio portrait, shot with a Canon EOS R5, 85mm lens, f/1.8, bokeh effect, dramatic soft natural volumetric lighting, golden hour.")
negative_prompt =("blurry, distorted, ugly, tiling, poorly drawn, out of frame, out of focus, duplicate, watermark, signature, text, jpeg artifacts, bad anatomy, deformed, extra limbs, extra fingers, malformed hands, missing limbs, disconnected limbs")

In [6]:
generator = torch.Generator(device=device).manual_seed(42) # set seed for reproducibility

image = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=24,  # increase for slightly higher quality
    guidance_scale=4.5,  # 3.5-6 is typical
    height=512,
    width=512,
    generator=generator
).images[0] # Access the first image in the list

# Save the result
image.save("face.png")
print("Saved to face.png")

  0%|          | 0/24 [00:00<?, ?it/s]

Saved to face.png
